In [1]:
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib as plt

%matplotlib inline

Database connection

In [2]:
from sqlalchemy import create_engine
POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'notebook'
POSTGRES_PASSWORD = 'postgres'
POSTGRES_DBNAME = 'postgres'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
cnx = create_engine(postgres_str)

From PLUTO, calculate LotFront * LotDepth, and compare it to LotArea, within a threshold of 100 square feet. If LotFront * LotDepth is not within that threshold, compare it to ArcGIS's Shape_Area within a threshold of 100 square feet. If LotFront * LotDepth is within the 100 square feet threshold, the LotArea on the record is a candidate for Shape_Area replacement. Result set is sorted by the difference between the Shape_Area and the LotFront * LotDepth, so the best candidates will appear first.

In [6]:
pd.read_sql_query('''SELECT CAST("BBL" AS VARCHAR),
"LotDepth",
"LotFront",
"LotArea",
"LotFront" * "LotDepth" AS "Front x Depth",
"Shape_Area",
ABS("Shape_Area" - ("LotFront" * "LotDepth")) AS "Shape_Area / Front x Depth Difference (Absolute value)",
"IrrLotCode"
FROM dcp.pluto192
WHERE (ROUND("LotDepth" * "LotFront") < "LotArea" - 100
OR ROUND("LotDepth" * "LotFront") > "LotArea" + 100)
AND (ROUND("LotDepth" * "LotFront") > (ROUND("Shape_Area") - 100)
AND (ROUND("LotDepth" * "LotFront") < (ROUND("Shape_Area") + 100)))
ORDER BY 7;''', cnx)

BBL  LotDepth  LotFront  LotArea  Front x Depth   Shape_Area  \
0     2049540069     75.00     17.00     1173      1275.0000  1274.998400   
1     5056870174    121.00     24.00     3025      2904.0000  2904.001901   
2     5057100001    100.00     55.00     6000      5500.0000  5500.002657   
3     5035720015    100.00     20.00     2650      2000.0000  2000.003055   
4     3044520216    100.00     23.92     3061      2392.0000  2392.007334   
...          ...       ...       ...      ...            ...          ...   
9236  3041010043     37.83     50.42     1650      1907.3886  1807.784834   
9237  2049730005    112.17     25.42     2750      2851.3614  2751.738143   
9238  2043300038     85.60     47.55     3800      4070.2800  3970.523715   
9239  1022150356    133.75     28.67     2069      3834.6125  3934.413748   
9240  4141500050     97.00     43.17     4006      4187.4900  4087.661841   

      Shape_Area / Front x Depth Difference (Absolute value) IrrLotCode  
0                                              0.001600               N  
1                                              0.001901               N  
2                                              0.002657               N  
3                                              0.003055               Y  
4                                              0.007334               N  
...                                                 ...             ...  
9236                                          99.603766               Y  
9237                                          99.623257               Y  
9238                                          99.756285               N  
9239                                          99.801248               Y  
9240                                          99.828159               Y  

[9241 rows x 8 columns]

Breakout report by LandUse code if needed.

In [5]:
pd.read_sql_query('''SELECT Differences."LandUse",
def.def AS "Definition",
Differences."LotArea Difference",
Counts."Records Affected"
FROM dcp.landuse_definitions def,
    (SELECT "LandUse",
      SUM("Shape_Area" - "LotArea") AS "LotArea Difference"
      FROM dcp.pluto192
      WHERE (ROUND("LotDepth" * "LotFront") < "LotArea" - 100
      OR ROUND("LotDepth" * "LotFront") > "LotArea" + 100)
      AND (ROUND("LotDepth" * "LotFront") > (ROUND("Shape_Area") - 100)
      AND (ROUND("LotDepth" * "LotFront") < (ROUND("Shape_Area") + 100)))
      GROUP BY "LandUse"
      ORDER BY "LandUse") AS Differences,
    (SELECT "LandUse", COUNT(*) AS "Records Affected"
      FROM dcp.pluto192
      WHERE (ROUND("LotDepth" * "LotFront") < "LotArea" - 100
      OR ROUND("LotDepth" * "LotFront") > "LotArea" + 100)
      AND (ROUND("LotDepth" * "LotFront") > (ROUND("Shape_Area") - 100)
      AND (ROUND("LotDepth" * "LotFront") < (ROUND("Shape_Area") + 100)))
      GROUP BY "LandUse"
      ORDER BY "LandUse") AS Counts
WHERE Differences."LandUse" = Counts."LandUse"
AND Differences."LandUse" = def.landuse;''', cnx)

LandUse                          Definition  LotArea Difference  \
0       01        One and Two Family Buildings        10863.311072   
1       02       Multi-Family Walkup Buildings      -223590.512598   
2       03     Multi-Family Elevator Buildings        66194.182381   
3       04    Mixed Residential and Commercial         -566.955988   
4       05               Commercial and Office       160679.894933   
5       06        Industrial and Manufacturing        84889.597293   
6       07          Transportation and Utility        18844.900206   
7       08  Public Facilities and Institutions        61942.831169   
8       09   Open Space and Outdoor Recreation        -1128.543975   
9       10                  Parking Facilities        38391.987469   
10      11                         Vacant Land       -74422.359365   

    Records Affected  
0               4944  
1               1316  
2                218  
3               1080  
4                546  
5                166  
6                 65  
7                185  
8                 57  
9                162  
10               450